In [25]:
import os
import shutil

import torch
import torch.utils.data
# import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import argparse
import re
import pickle
from helpers import makedir
import protopnet
import push
import prune
import train_and_test as tnt
import save
from log import create_logger
from preprocess import mean, std, preprocess_input_function

In [12]:
# ORIGINAL WORK FOR REFERENCE
"""
parser = argparse.ArgumentParser()
parser.add_argument('-gpuid', nargs=1, type=str, default='0') # python3 main.py -gpuid=0,1,2,3
args = parser.parse_args()
os.environ['CUDA_VISIBLE_DEVICES'] = args.gpuid[0]
print(os.environ['CUDA_VISIBLE_DEVICES'])
"""

"\nparser = argparse.ArgumentParser()\nparser.add_argument('-gpuid', nargs=1, type=str, default='0') # python3 main.py -gpuid=0,1,2,3\nargs = parser.parse_args()\nos.environ['CUDA_VISIBLE_DEVICES'] = args.gpuid[0]\nprint(os.environ['CUDA_VISIBLE_DEVICES'])\n"

In [13]:
# ORIGINAL WORK, SAVING DATA
"""
from settings import base_architecture, img_size, prototype_shape, num_classes, \
                     prototype_activation_function, add_on_layers_type, experiment_run

base_architecture_type = re.match('^[a-z]*', base_architecture).group(0)

model_dir = './saved_models/' + base_architecture + '/' + experiment_run + '/'
makedir(model_dir)
shutil.copy(src=os.path.join(os.getcwd(), __file__), dst=model_dir)
shutil.copy(src=os.path.join(os.getcwd(), 'settings.py'), dst=model_dir)
shutil.copy(src=os.path.join(os.getcwd(), base_architecture_type + '_features.py'), dst=model_dir)
shutil.copy(src=os.path.join(os.getcwd(), 'model.py'), dst=model_dir)
shutil.copy(src=os.path.join(os.getcwd(), 'train_and_test.py'), dst=model_dir)
"""

"\nfrom settings import base_architecture, img_size, prototype_shape, num_classes,                      prototype_activation_function, add_on_layers_type, experiment_run\n\nbase_architecture_type = re.match('^[a-z]*', base_architecture).group(0)\n\nmodel_dir = './saved_models/' + base_architecture + '/' + experiment_run + '/'\nmakedir(model_dir)\nshutil.copy(src=os.path.join(os.getcwd(), __file__), dst=model_dir)\nshutil.copy(src=os.path.join(os.getcwd(), 'settings.py'), dst=model_dir)\nshutil.copy(src=os.path.join(os.getcwd(), base_architecture_type + '_features.py'), dst=model_dir)\nshutil.copy(src=os.path.join(os.getcwd(), 'model.py'), dst=model_dir)\nshutil.copy(src=os.path.join(os.getcwd(), 'train_and_test.py'), dst=model_dir)\n"

In [14]:
# ORIGINAL WORK, ONLY FOR REFERENCE
# load the data
"""
experiment_run = '003'
data_path = './datasets/cub200_cropped/'
train_dir = data_path + 'train_cropped_augmented/'
test_dir = data_path + 'test_cropped/'
train_push_dir = data_path + 'train_cropped/'

train_batch_size = 80
test_batch_size = 100
train_push_batch_size = 75

normalize = transforms.Normalize(mean=mean,
                                 std=std)

# all datasets
# train set
train_dataset = datasets.ImageFolder(
    train_dir,
    transforms.Compose([
        transforms.Resize(size=(img_size, img_size)),
        transforms.ToTensor(),
        normalize,
    ]))
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=train_batch_size, shuffle=True,
    num_workers=4, pin_memory=False)
# push set
train_push_dataset = datasets.ImageFolder(
    train_push_dir,
    transforms.Compose([
        transforms.Resize(size=(img_size, img_size)),
        transforms.ToTensor(),
    ]))

train_push_loader = torch.utils.data.DataLoader(
    train_push_dataset, batch_size=train_push_batch_size, shuffle=False,
    num_workers=4, pin_memory=False)

# test set
test_dataset = datasets.ImageFolder(
    test_dir,
    transforms.Compose([
        transforms.Resize(size=(img_size, img_size)),
        transforms.ToTensor(),
        normalize,
    ]))
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=test_batch_size, shuffle=False,
    num_workers=4, pin_memory=False)
"""

"\nexperiment_run = '003'\ndata_path = './datasets/cub200_cropped/'\ntrain_dir = data_path + 'train_cropped_augmented/'\ntest_dir = data_path + 'test_cropped/'\ntrain_push_dir = data_path + 'train_cropped/'\n\ntrain_batch_size = 80\ntest_batch_size = 100\ntrain_push_batch_size = 75\n\nnormalize = transforms.Normalize(mean=mean,\n                                 std=std)\n\n# all datasets\n# train set\ntrain_dataset = datasets.ImageFolder(\n    train_dir,\n    transforms.Compose([\n        transforms.Resize(size=(img_size, img_size)),\n        transforms.ToTensor(),\n        normalize,\n    ]))\ntrain_loader = torch.utils.data.DataLoader(\n    train_dataset, batch_size=train_batch_size, shuffle=True,\n    num_workers=4, pin_memory=False)\n# push set\ntrain_push_dataset = datasets.ImageFolder(\n    train_push_dir,\n    transforms.Compose([\n        transforms.Resize(size=(img_size, img_size)),\n        transforms.ToTensor(),\n    ]))\n\ntrain_push_loader = torch.utils.data.DataLoader(\n 

In [30]:
# OURS

# SETTINGS FILE

# our default architecture
# it is used in ppnet class to download pretrained resnet34
base_architecture = 'resnet34'
img_size = 224

# 4 prototypes per 1 class, 20 classes == 80 prototypes
prototype_shape = (80, 128, 1, 1)
num_classes = 20
prototype_activation_function = 'log'
add_on_layers_type = 'regular'

train_batch_size = 80
test_batch_size = 100
train_push_batch_size = 75



from dataset.CombinedDataLoader import CDL
root = './dataset/data'

normalize = transforms.Normalize(mean=mean,
                                 std=std)

trans = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize # very important
])

trans_push = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    # don't normalize push loader!
])


ds_train = CDL(root, train=True, transform=trans, num_classes=num_classes, images=['cropped', "rotated", "sheared", "skewed"])
ds_train_push = CDL(root, train=True, transform=trans_push, num_classes=num_classes, images=['cropped', "rotated", "sheared", "skewed"])
ds_test = CDL(root, train=False, transform=trans, num_classes=num_classes, images=['cropped'])

In [31]:
train_push_loader = torch.utils.data.DataLoader(ds_train_push, batch_size=train_push_batch_size, shuffle=True, num_workers=2)

train_loader = torch.utils.data.DataLoader(ds_train, batch_size=train_batch_size, shuffle=True, num_workers=2)

test_loader = torch.utils.data.DataLoader(ds_test, batch_size=test_batch_size, shuffle=False, num_workers=2)

In [32]:
from datetime import datetime
now = datetime.now()
dt_string = now.strftime("%d_%m_%Y_%H_%M_%S")
model_dir = f"ProtoPNet_base_arch_{base_architecture}_n_classes_{num_classes}_prot_shape_{prototype_shape}_{dt_string}"
makedir(model_dir)

log, logclose = create_logger(log_filename=os.path.join(model_dir, 'train.log'))
img_dir = os.path.join(model_dir, 'img')
makedir(img_dir)
weight_matrix_filename = 'outputL_weights'
prototype_img_filename_prefix = 'prototype-img'
prototype_self_act_filename_prefix = 'prototype-self-act'
proto_bound_boxes_filename_prefix = 'bb'

In [33]:
log('training set size: {0}'.format(len(train_loader.dataset)))
log('push set size: {0}'.format(len(train_push_loader.dataset)))
log('test set size: {0}'.format(len(test_loader.dataset)))
log('train batch size: {0}'.format(train_batch_size))

# print(len(train_loader.dataset))
# print(len(train_push_loader.dataset))
# print(len(test_loader.dataset))
# print(train_batch_size)

# construct the model
ppnet = protopnet.construct_PPNet(base_architecture=base_architecture,
                              pretrained=True, img_size=img_size,
                              prototype_shape=prototype_shape,
                              num_classes=num_classes,
                              prototype_activation_function=prototype_activation_function,
                              add_on_layers_type=add_on_layers_type)

ppnet = ppnet.cuda()
ppnet_multi = torch.nn.DataParallel(ppnet)
class_specific = True

training set size: 2400
push set size: 2400
test set size: 515
train batch size: 80


In [34]:
# define optimizer

joint_optimizer_lrs = {'features': 1e-4,
                       'add_on_layers': 3e-3,
                       'prototype_vectors': 3e-3}
joint_lr_step_size = 5

warm_optimizer_lrs = {'add_on_layers': 3e-3,
                      'prototype_vectors': 3e-3}

last_layer_optimizer_lr = 1e-4

coefs = {
    'crs_ent': 1,
    'clst': 0.8,
    'sep': -0.08,
    'l1': 1e-4,
}

joint_optimizer_specs = \
[{'params': ppnet.features.parameters(), 'lr': joint_optimizer_lrs['features'], 'weight_decay': 1e-3}, # bias are now also being regularized
 {'params': ppnet.add_on_layers.parameters(), 'lr': joint_optimizer_lrs['add_on_layers'], 'weight_decay': 1e-3},
 {'params': ppnet.prototype_vectors, 'lr': joint_optimizer_lrs['prototype_vectors']},
]
joint_optimizer = torch.optim.Adam(joint_optimizer_specs)
joint_lr_scheduler = torch.optim.lr_scheduler.StepLR(joint_optimizer, step_size=joint_lr_step_size, gamma=0.1)


warm_optimizer_specs = \
[{'params': ppnet.add_on_layers.parameters(), 'lr': warm_optimizer_lrs['add_on_layers'], 'weight_decay': 1e-3},
 {'params': ppnet.prototype_vectors, 'lr': warm_optimizer_lrs['prototype_vectors']},
]
warm_optimizer = torch.optim.Adam(warm_optimizer_specs)


last_layer_optimizer_specs = [{'params': ppnet.last_layer.parameters(), 'lr': last_layer_optimizer_lr}]
last_layer_optimizer = torch.optim.Adam(last_layer_optimizer_specs)

# weighting of different training losses



# number of training epochs, number of warm epochs, push start epoch, push epochs
num_train_epochs = 500
num_warm_epochs = 5
push_start = 10
push_epochs = [i for i in range(num_train_epochs) if i % 10 == 0]

In [35]:
def load_model(model_dir=None, model_name=None, ppnet=None, joint_optimizer=None, warm_optimizer=None, last_layer_optimizer=None):
    # none everywhere as to not load anything that jupyter holds in memory
    model_path = model_dir + '/' + model_name
    joint_optimizer_path = model_dir + '/joint_optimizer.pth'
    warm_optimizer_path = model_dir + '/warm_optimizer.pth'
    last_layer_optimizer_path = model_dir + '/last_layer_optimizer.pth'
    variables_path = model_dir + '/variables.pkl'


    # WARNING: THEY SAVE WHOLE MODEL, I SAVE DICT STATE
    ppnet.load_state_dict(torch.load(model_path))
    
    # Load the optimizer states
    joint_optimizer.load_state_dict(torch.load(joint_optimizer_path))
    warm_optimizer.load_state_dict(torch.load(warm_optimizer_path))
    last_layer_optimizer.load_state_dict(torch.load(last_layer_optimizer_path))
    
    # It probably isn't that useful, so they are not returned
    with open(variables_path, 'rb') as f:
        variables = pickle.load(f)
    coefs = variables['coefs']
    num_train_epochs = variables['num_train_epochs']
    num_warm_epochs = variables['num_warm_epochs']
    push_start = variables['push_start']
    push_epochs = variables['push_epochs']
    class_specific = variables['class_specific']


    
    # Load other variables as needed
    log(f"Model and optimizers loaded from {model_dir}")
    log(f"Model: {model_path}")
    log(f"Joint optimizer: {joint_optimizer_path}")
    log(f"Warm optimizer: {warm_optimizer_path}")
    log(f"Last layer_optimizer: {last_layer_optimizer_path}")
    # log(f"Variables: {variables_path}")

In [36]:
load_model_dir = 'ProtoPNet_base_arch_resnet34_n_classes_20_prot_shape_(80, 128, 1, 1)_28_05_2023_15_15_19'
load_model_name = 'ProtoPNet_base_arch_resnet34_n_classes_20_prot_shape_(80, 128, 1, 1)_28_05_2023_15_15_19_0.8019417475728156.pth'
load_model(model_dir=load_model_dir, model_name=load_model_name, ppnet=ppnet, joint_optimizer=joint_optimizer, warm_optimizer=warm_optimizer, last_layer_optimizer=last_layer_optimizer)

Model and optimizers loaded from logdir_4_parts_0
Model: logdir_4_parts_0/logdir_4_parts_0_0.8058252427184466.pth
Joint optimizer: logdir_4_parts_0/joint_optimizer.pth
Warm optimizer: logdir_4_parts_0/warm_optimizer.pth
Last layer_optimizer: logdir_4_parts_0/last_layer_optimizer.pth


In [ ]:
# train the model
target_acc = 0.80
log('start training')
import copy
for epoch in range(num_train_epochs):
    log('epoch: \t{0}'.format(epoch))

    if epoch < num_warm_epochs:
        tnt.warm_only(model=ppnet_multi, log=log)
        _ = tnt.train(model=ppnet_multi, dataloader=train_loader, optimizer=warm_optimizer,
                      class_specific=class_specific, coefs=coefs, log=log)
    else:
        tnt.joint(model=ppnet_multi, log=log)
        joint_lr_scheduler.step()
        _ = tnt.train(model=ppnet_multi, dataloader=train_loader, optimizer=joint_optimizer,
                      class_specific=class_specific, coefs=coefs, log=log)

    accu = tnt.test(model=ppnet_multi, dataloader=test_loader,
                    class_specific=class_specific, log=log)
    save.save_model_w_condition(model=ppnet, model_dir=model_dir, model_name=str(epoch) + 'nopush', accu=accu,
                                target_accu=target_acc, log=log)

    if epoch >= push_start and epoch in push_epochs:
        push.push_prototypes(
            train_push_loader, # pytorch dataloader (must be unnormalized in [0,1])
            prototype_network_parallel=ppnet_multi, # pytorch network with prototype_vectors
            class_specific=class_specific,
            preprocess_input_function=preprocess_input_function, # normalize if needed
            prototype_layer_stride=1,
            root_dir_for_saving_prototypes=img_dir, # if not None, prototypes will be saved here
            epoch_number=epoch, # if not provided, prototypes saved previously will be overwritten
            prototype_img_filename_prefix=prototype_img_filename_prefix,
            prototype_self_act_filename_prefix=prototype_self_act_filename_prefix,
            proto_bound_boxes_filename_prefix=proto_bound_boxes_filename_prefix,
            save_prototype_class_identity=True,
            log=log)
        accu = tnt.test(model=ppnet_multi, dataloader=test_loader,
                        class_specific=class_specific, log=log)
        save.save_model_w_condition(model=ppnet, model_dir=model_dir, model_name=str(epoch) + 'push', accu=accu,
                                    target_accu=target_acc, log=log)

        if prototype_activation_function != 'linear':
            tnt.last_only(model=ppnet_multi, log=log)
            for i in range(20):
                log('iteration: \t{0}'.format(i))
                _ = tnt.train(model=ppnet_multi, dataloader=train_loader, optimizer=last_layer_optimizer,
                              class_specific=class_specific, coefs=coefs, log=log)
                accu = tnt.test(model=ppnet_multi, dataloader=test_loader,
                                class_specific=class_specific, log=log)
                save.save_model_w_condition(model=ppnet, model_dir=model_dir, model_name=str(epoch) + '_' + str(i) + 'push', accu=accu,
                                            target_accu=target_acc, log=log)
   
logclose()


In [38]:
# SAVING MODEL TO TRAIN LATER

# Save the model state
model_path = model_dir + '/' + model_dir + '_' + str(accu) + '.pth'
torch.save(ppnet.state_dict(), model_path)

# Save the optimizer states
joint_optimizer_path = model_dir + '/joint_optimizer.pth'
torch.save(joint_optimizer.state_dict(), joint_optimizer_path)

warm_optimizer_path = model_dir + '/warm_optimizer.pth'
torch.save(warm_optimizer.state_dict(), warm_optimizer_path)

last_layer_optimizer_path = model_dir + '/last_layer_optimizer.pth'
torch.save(last_layer_optimizer.state_dict(), last_layer_optimizer_path)

# Save other variables
variables_path = model_dir + '/variables.pkl'
variables = {
    'coefs': coefs,
    'num_train_epochs': num_train_epochs,
    'num_warm_epochs': num_warm_epochs,
    'push_start': push_start,
    'push_epochs': push_epochs,
    'class_specific': class_specific,
    # Include other variables you want to save
}
with open(variables_path, 'wb') as f:
    pickle.dump(variables, f)